# Kaggle

In [1]:
import os
import numpy as np
import cv2
import time

import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision
from mediapipe import solutions
from mediapipe.framework.formats import landmark_pb2
import matplotlib.pyplot as plt
from PIL import Image
from scipy import stats as st

### 1. Hand Detection using MediaPipe

In [ ]:
def detect_hand(path):
    mp_hands = mp.solutions.hands
    hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)

    image = cv2.imread(path)
    
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert the image to RGB.

    results = hands.process(image_rgb)

    if results.multi_hand_landmarks:

        h, w, c = image.shape 
        hand_landmarks = results.multi_hand_landmarks[0] 
        x_coords = [lm.x * w for lm in hand_landmarks.landmark]
        y_coords = [lm.y * h for lm in hand_landmarks.landmark]
        
        x_min = int(min(x_coords))
        x_max = int(max(x_coords))
        y_min = int(min(y_coords))
        y_max = int(max(y_coords))

        margin = 150
        hand_image = image[y_min-margin:y_max+margin, x_min-margin:x_max+margin]

        hands.close()
        return hand_image

    else:
        print("No hand detected.")
        hands.close()
        return None
    


### 2. Hand Processing using detect_hand

In [ ]:
def get_mask_color(hand_image,v=30):
    """
    This is the original and more complex function for obtaining the mask of the hand. This function will be used to transform images
    external to the dataset into photos similar to theones it used to train the model.
    """
    bg = np.array(hand_image[0][:20])
    analysis = st.mode(bg)

    # Find the most popular color in the first 20
    popular_color = analysis.mode

    mask = np.ones((hand_image.shape[0],hand_image.shape[1]),dtype=np.uint8)
    
    high_cota = [min(255,popular_color[i]+v) for i in range(3)]
    low_cota = [max(0,popular_color[i]-v) for i in range(3)]
    for y,row in enumerate(hand_image):
        for x,px in enumerate(row):
            if (px[0]<=high_cota[0] and px[1]<=high_cota[1] and px[2]<=high_cota[2]) \
             and (px[0]>=low_cota[0] and px[1]>=low_cota[1] and px[2]>=low_cota[2]):
                mask[y][x] = 0
    return mask

In [4]:
def hand_processing(path):
    hand_image = detect_hand(path)
    if hand_image is not None:
        if hand_image.shape[0] > 0:
            mask = get_mask_color(hand_image, 30)

            background_mask = cv2.bitwise_not(mask)
            black_background = np.zeros_like(hand_image)

            foreground = cv2.bitwise_and(hand_image, hand_image, mask=mask)
            background = cv2.bitwise_and(black_background, black_background, mask=background_mask)

            result = cv2.add(foreground, background)

            height, width, _ = result.shape
            square_size = max(height, width)

            square_image = np.zeros((square_size, square_size, 3), dtype=np.uint8)
            y_offset = (square_size - height) // 2
            x_offset = (square_size - width) // 2
            square_image[y_offset:y_offset + height, x_offset:x_offset + width] = result
            resized_image = cv2.resize(square_image, (400, 400), interpolation=cv2.INTER_AREA)

            
            print('Hand detected in', path)
            return resized_image

    else:
        #print('No hand detected in', filename)
        return None

### 3. Frame extraction and Hand Processing Application

In [ ]:
import cv2
import os


base_folder = 'Images/' # Folder where the video is located
video_name = 'cat' # Name of the video
formato = '.mp4' # Format of the video
final_folder = base_folder+"Results/" # Folder where the frames will be saved
vidcap = cv2.VideoCapture('Images/'+video_name+formato)
success,image = vidcap.read()
count = 0
ratio = 2

while success: # For all valid frames in the video, extract the hand and save it in the final folder
    if count % ratio == 0:
        cv2.imwrite(final_folder+"frame%d.jpg" % count, image)  # frame en carpeta
    success,image = vidcap.read()
    print('Read a new frame: ', success)
    count += 1


In [ ]:
# For every file in the folder try detecting its hand using the code above
frames = []
for filename in os.listdir(final_folder):
    frames.append(hand_processing(os.path.join(final_folder, filename)))

In [8]:
for i,frame in enumerate(frames):
    img_resized = Image.fromarray(frame)
    img_resized.save(f"frame{i}_processed.png")